Импортируем библиотеки

In [2]:
from bs4 import BeautifulSoup
import requests

Подключим гугл диск

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


С помощью CURL конвертера, вставим печеньки и заголовки

In [7]:
import requests

cookies = {
    '__ddg1_': 'fsosHW7JdV7omHDqd4Li',
    'jtnews_ab_24': 'A',
    'jtnews_ab_29': 'A',
    'ngs_uid': 'wxPcG2QTDIqj7sLFA+YgAg==',
    'ngs_analytics_uid': 'wxPcEmQTDIofu1EALCSNAg==',
    '_ym_uid': '1678969996505315255',
    '_ym_d': '1678969996',
    '_ym_isad': '2',
    '_ym_visorc': 'b',
    'adrdel': '1',
    'adrcid': 'ADqjiWbDphRyVbWb1t7cQvA',
}

headers = {
    'authority': 'ufa1.ru',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'ru,en;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': '__ddg1_=fsosHW7JdV7omHDqd4Li; jtnews_ab_24=A; jtnews_ab_29=A; ngs_uid=wxPcG2QTDIqj7sLFA+YgAg==; ngs_analytics_uid=wxPcEmQTDIofu1EALCSNAg==; _ym_uid=1678969996505315255; _ym_d=1678969996; _ym_isad=2; _ym_visorc=b; adrdel=1; adrcid=ADqjiWbDphRyVbWb1t7cQvA',
    'sec-ch-ua': '"Not?A_Brand";v="8", "Chromium";v="108", "Yandex";v="23"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 YaBrowser/23.1.5.708 Yowser/2.5 Safari/537.36',
}

params = {
    'page': '1',
}

response = requests.get('https://ufa1.ru/text/', params=params, cookies=cookies, headers=headers)

Создадим колонки для наших новостей и их характеристик, такие как:

Название, Ссылка, Рубрика, Дата и время публикации, Количество просмотров - интерес публики

In [66]:
import time
from random import randint

col_name_news = []
col_links = []
col_rubrics = []
col_datetime = []
col_views = []

for page in range(1, 150):
  time.sleep(randint(3, 6))  # и сайт прогрузится, и систему обманем

  r = requests.get(f'https://ufa1.ru/text/?page={page}', cookies=cookies, headers=headers)
  soup = BeautifulSoup(r.text)

  name_news = [item.text.strip() for item in soup.find_all('h2', {'class': 'h9Jmx'})]
  col_name_news.extend(name_news)

  links = ['https://ufa1.ru' + item.get('href') for item in soup.find_all('a', {'data-test': 'archive-record-header'})]
  col_links.extend(links)

  rubrics = [item.text.strip() for item in soup.find_all('div', {'data-test': 'archive-record-rubrics'})]
  col_rubrics.extend(rubrics)

  dates = [item['datetime'].replace('T', ' ') for item in soup.find_all('time')]
  col_datetime.extend(dates)

  views = [int(item.find('span').text.replace('\xa0', '')) for item in soup.find_all('div', {'data-test': 'record-stats-view'})]
  col_views.extend(views)


print(len(col_name_news))
print(len(col_links))
print(len(col_rubrics))
print(len(col_datetime))
print(len(col_views))

5960
5960
5960
5960
5960


Переведем рубрики из качественных данных в количественные

In [73]:
col_rubrics_copy = col_rubrics.copy()
# del col_rubrics_copy[0]  # удаляем пустую строку
col_rubrics_for_set = [item.split('  ')[0].strip() for item in col_rubrics_copy]
col_rubrics_set = set(col_rubrics_for_set)  # оставляем по одному экземпляру рубрики для дальнейшей нумерации
col_rubrics_list = list(col_rubrics_set)
info_rubrics = [[item, index] for item, index in enumerate(list(col_rubrics_set))]  # нумеруем рубрики для составления количественной колонки 
print(info_rubrics)

# col_num_rubrics = [col_rubrics_list.index(item.split('  ')[0].strip()) for item in col_rubrics_copy]
col_num_rubrics = []
for item in col_rubrics_copy:
  if item == '':
    col_num_rubrics.append(0)  # Для новостей без рубрики
  else:
    col_num_rubrics.append(int(col_rubrics_list.index(item.split('  ')[0].strip())))

print(col_num_rubrics)

[[0, ''], [1, 'Спорт'], [2, 'Экономика'], [3, 'Еда'], [4, 'Недвижимость'], [5, 'Животные'], [6, 'Культура'], [7, 'Стиль и красота'], [8, 'Авто'], [9, 'Осень'], [10, 'Наука'], [11, 'Лето'], [12, 'Город'], [13, 'Зима'], [14, 'Мой дом'], [15, 'Экология'], [16, 'Образование'], [17, 'Криминал'], [18, 'Страна и мир'], [19, 'Дороги и транспорт'], [20, 'Работа'], [21, 'Он и она'], [22, 'Политика'], [23, 'Здоровье'], [24, 'Развлечения'], [25, 'Происшествия'], [26, 'Семья'], [27, 'Спецпроекты'], [28, 'Религия'], [29, 'Бизнес'], [30, 'Весна']]
[12, 2, 25, 22, 25, 18, 28, 12, 7, 18, 6, 22, 25, 29, 1, 7, 25, 12, 19, 18, 6, 19, 23, 25, 17, 16, 12, 25, 25, 1, 25, 19, 20, 18, 16, 25, 25, 26, 19, 25, 25, 23, 22, 18, 25, 25, 12, 3, 25, 23, 12, 25, 23, 12, 26, 18, 12, 12, 25, 25, 23, 25, 12, 12, 29, 12, 30, 2, 23, 23, 30, 25, 29, 18, 25, 25, 17, 17, 20, 23, 23, 12, 19, 12, 25, 18, 22, 29, 18, 29, 18, 18, 29, 24, 2, 22, 25, 12, 25, 24, 25, 17, 23, 25, 12, 12, 18, 25, 12, 16, 14, 18, 25, 12, 24, 24, 6, 25,

Создадим еще одну количественную колонку. В данной колонке будет период между предыдущей публикацией

In [80]:
from datetime import datetime

patt = '%Y-%m-%d %H:%M:%S'
dt_now = [datetime.now()]

col_datetime_datetime = [datetime.strptime(item, patt) for item in col_datetime]
tmp_datetime_datetime = dt_now + col_datetime_datetime
col_period = []
for i in range(len(tmp_datetime_datetime)):
  if i == 0:
    continue
  else:
    col_period.append((tmp_datetime_datetime[i] - tmp_datetime_datetime[i-1]).seconds)
print(len(col_period))
print(col_period)  # время от времени выходят по две новости одновременно, поэтому нулевые значение, но 0 это тоже показатель частоты

5960
[14507, 86100, 84900, 85320, 84900, 83280, 84000, 0, 56400, 83100, 86100, 82800, 85509, 86124, 83967, 0, 86100, 83100, 85800, 84108, 83932, 86060, 84900, 84420, 85380, 85800, 85800, 85200, 85380, 85020, 85791, 85509, 85894, 85778, 85428, 85440, 86160, 85200, 85800, 84480, 85320, 85800, 85380, 83820, 0, 84000, 85800, 85800, 83400, 85800, 0, 85200, 84000, 85800, 84600, 53100, 84360, 82860, 84540, 84420, 83940, 84480, 84034, 85106, 85560, 85320, 83373, 83683, 86024, 86280, 84886, 84134, 86100, 83400, 85200, 84600, 84720, 85380, 84900, 84840, 84360, 0, 84900, 85500, 85200, 82800, 85800, 84600, 70939, 71861, 85320, 84180, 86100, 0, 85726, 86054, 83822, 82798, 82800, 81900, 85500, 85620, 85380, 84600, 85320, 85680, 84480, 85320, 84600, 84600, 83100, 85500, 85380, 81720, 84900, 85500, 85980, 83220, 84300, 85800, 82800, 85800, 84600, 56400, 83100, 86100, 82800, 82800, 85380, 85620, 0, 84600, 85440, 83760, 85200, 84600, 85800, 85200, 85260, 86040, 85500, 83700, 85500, 0, 84600, 85560, 8634

Преобразуем всё в ДатаФрейм

In [82]:
import pandas as pd

for_df = {'Заголовок новости': col_name_news, 'Ссылка': col_links, 'Рубрика': col_rubrics, 'Дата публикации': col_datetime, 'Количество просмотров': col_views, 'Частота публикаций': col_period, 'Метка рубрики': col_num_rubrics}
df = pd.DataFrame.from_dict(for_df)

df.head(10)

,Заголовок новости,Ссылка,Рубрика,Дата публикации,Количество просмотров,Частота публикаций,Метка рубрики
0,В мэрии Уфы пообещали благоустроить площадь пе...,https://ufa1.ru/text/gorod/2023/03/17/72139637/,Город,2023-03-17 10:05:00,214,14507,12
1,"Верните свои деньги: объясняем в картинках, ка...",https://ufa1.ru/text/economics/2023/03/17/7213...,Экономика,2023-03-17 10:00:00,148,86100,2
2,Прокуратура Башкирии проверит работу газовиков...,https://ufa1.ru/text/incidents/2023/03/17/7213...,Происшествия,2023-03-17 09:35:00,412,84900,25
3,В Башкирии погиб из-за схода снега замглавы ад...,https://ufa1.ru/text/incidents/2023/03/17/7213...,Политика Происшествия,2023-03-17 09:17:00,1991,85320,22
4,В Башкирии в жилом доме произошел взрыв газа —...,https://ufa1.ru/text/incidents/2023/03/17/7213...,Происшествия,2023-03-17 08:52:00,1996,84900,25
5,В России разносят повестки «для уточнения данн...,https://ufa1.ru/text/world/2023/03/17/72139061/,Страна и мир,2023-03-17 08:00:00,870,83280,18
6,"Не «Ар-Рахимом» единым: смотрим, какие еще меч...",https://ufa1.ru/text/religion/2023/03/17/72137...,Религия,2023-03-17 07:20:00,1619,84000,28
7,«Сразу всех не будут же выгонять»: как продавц...,https://ufa1.ru/text/gorod/2023/03/17/72138098/,Город,2023-03-17 07:20:00,1659,0,12
8,«Надо сначала создать красивое тело»: блогер-и...,https://ufa1.ru/text/style/2023/03/16/72136613/,Стиль и красота,2023-03-16 23:00:00,729,56400,7
9,Депутата признали виновным в дискредитации ВС ...,https://ufa1.ru/text/world/2023/03/16/72138092/,Страна и мир,2023-03-16 22:05:00,1044,83100,18


Сохраним ДатаФрейм в файл

In [83]:
df.to_csv('news_data.csv', index=False)

Сделаем датасет, для этого импортируем дополнительные библиотеки/функции

In [95]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

Загрузим данные из датафрейма

In [96]:
data = pd.read_csv("news_data.csv")

Выберем столбцы, которые будем использовать в качестве признаков

In [112]:
features = ['Количество просмотров', 'Частота публикаций']

Выделим целевую переменную, пусть это будет Рубрика, т.е. колнка "Метка рубрики"

In [113]:
target = data['Метка рубрики']

Применим Label Encoding, чтобы преобразовать текстовые значения в числовые

In [114]:
le = LabelEncoder()
target = le.fit_transform(target)

Преобразуем значения признаков в числовой формат

In [115]:
data[features] = data[features].apply(pd.to_numeric, errors='coerce')

Разделим выборку на обучающую и тестовую

In [116]:
X_train, X_test, y_train, y_test = train_test_split(data[features], target, test_size=0.2, random_state=0)

Создадим датасет путем объединения матрицы признаков и вектора целевых значений

In [118]:
dataset = pd.concat([X_train, pd.DataFrame(y_train, columns=['Метка_рубрики'])], axis=1)

Сохраним датасет в файл

In [119]:
dataset.to_csv('my_dataset.csv', index=False)

###Созданный датасет, можно использовать для анализа частотности определенных новостей, а так же класификацию по рубрикам.
Метки рубрик маркеруются следующим образом:

[0, 'Без рубрики'], 

[1, 'Спорт'],

[2, 'Экономика'],

[3, 'Еда'], 

[4, 'Недвижимость'], 

[5, 'Животные'],

[6, 'Культура'], 

[7, 'Стиль и красота'], 

[8, 'Авто'], 

[9, 'Осень'], 

[10, 'Наука'], 

[11, 'Лето'], 

[12, 'Город'], 

[13, 'Зима'], 

[14, 'Мой дом'], 

[15, 'Экология'], 

[16, 'Образование'], 

[17, 'Криминал'], 

[18, 'Страна и мир'], 

[19, 'Дороги и транспорт'],

[20, 'Работа'], 

[21, 'Он и она'], 

[22, 'Политика'], 

[23, 'Здоровье'], 

[24, 'Развлечения'], 

[25, 'Происшествия'], 

[26, 'Семья'], 

[27, 'Спецпроекты'], 

[28, 'Религия'], 

[29, 'Бизнес'], 

[30, 'Весна']